In [1]:
import pickle
import os

In [2]:
with open(os.path.join("data","yelp_2017","dict.pkl"),"rb") as f:
    word_to_ind = pickle.load(f,encoding="bytes")
with open(os.path.join("data","yelp_2017","word_emb.pkl"),"rb") as f:
    emb = pickle.load(f,encoding="bytes")

In [33]:
max(word_to_ind.values()),emb.shape

(49999, (50000, 64))

In [73]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    
        
# conn.close()

In [12]:
# x=Update("create view RevAB_CA as select \
# User.id as user_id, User.review_count as user_review_count, User.average_rating as user_rating, \
# Business.city as city, Business.state as state, Business.ratings as item_rating, Business.id as item_id, \
# Reviews.id as review_id, Reviews.ts as ts, Reviews.rating as review_rating \
# from Reviews \
# Inner JOIN User on User.id = Reviews.user_id \
# INNER JOIN Business on Business.id = Reviews.business_id \
# where Business.city = \"Las Vegas\"")

In [75]:
data = Select("Select * from RevAB_CA where user_review_count > 15 LIMIT 10000")
#

In [76]:
query = data
data = []
for row in query:
    data += [row]

In [78]:
items = list(set([x[6] for x in data]))
users = list(set([x[0] for x in data]))
items_ind = {items[i]:i for i in range(len(items))}

users_ind = {users[i]:i for i in range(len(users))}

In [80]:
len(items),len(users),len(data[0])

(1015, 5661, 11)

In [87]:
from collections import defaultdict 
rev_u = defaultdict(lambda: defaultdict(dict))
rev_i = defaultdict(lambda: defaultdict(dict))
for review in data:
    user = review[0]
    item = review[6]
    text = review[10]#o
    rev_u [user][item] = text
    rev_i [item][user] = text
rev_u = dict(rev_u)
rev_i = dict(rev_i)

In [95]:
words = set(word_to_ind.keys())
wordList = list(words)

In [149]:
import difflib
import numpy as np
filename = 'glove.6B.50d.txt'
def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    line = None
#     try:
    with open(filename) as f:
        for line in f:
            row = line.strip().split(' ')
            print(row[0],)
            vocab.append(row[0])
            embd.append(row[1:])
    print('Loaded GloVe!')
#     except Exception as e:
#         print(line)
    file.close()
    return vocab,embd
vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
vocab_set = set(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)


                    

the
,
.
of
to
and
in
a
"
's
for
-
that
on
is
was
said
with
he
as
it
by
at
(
)
from
his
''
``
an
be
has
are
have
but
were
not
this
who
they
had
i
which
will
their
:
or
its
one
after
new
been
also
we
would
two
more
'
first
about
up
when
year
there
all
--
out
she
other
people
n't
her
percent
than
over
into
last
some
government
time
$
you
years
if
no
world
can
three
do
;
president
only
state
million
could
us
most
_
against
u.s.
so
them
what
him
united
during
before
may
since
many
while
where
states
because
now
city
made
like
between
did
just
national
day
country
under
such
second
then
company
group
any
through
china
four
being
down
war
back
off
south
american
minister
police
well
including
team
international
week
officials
still
both
even
high
part
told
those
end
former
these
make
billion
work
our
home
school
party
house
old
later
get
another
tuesday
news
long
five
called
1
wednesday
military
way
used
much
next
monday
thursday
friday
game
here
?
should
take
very
my
north
security
season
yo

militants
holding
500
regular
required
lawyer
word
debt
guard
wounded
candidates
assembly
expressed
character
pro
fuel
martin
supreme
broke
ministers
everyone
actually
equipment
additional
sept.
denied
â€œ
friend
features
levels
approved


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2273: character maps to <undefined>

In [147]:
vocab[:5],vocab_size

(['the', ',', '.', 'of', 'to'], 1413)

In [60]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dense,Dropout,BatchNormalization, SimpleRNN, GRU, LSTM,Bidirectional,Conv2D,Merge,Flatten,MaxPooling2D

In [145]:
def preprocess(text,maxlen):
    text = text.split()
    count = 0
    _w = []
    for word in text:
        word = word.lower()
        if word in vocab_set:
            _w += [word]
        elif word[:-1] in vocab_set and word[-1] == ".":
            _w += [word]
            _w += [word[-1]]
        else:
            print(word[:-1])
            print(word)
    print(len(_w),len(text)),
    return _w

In [146]:
user = list(rev_u.keys())[0]
preprocess(" ".join([rev_u[user][item] for item in rev_u[user] ]),50)

cheb
cheba
hut!!
hut!!!
men
menu
disappointed
disappointed.
don'
don't
stone
stoned
subs
subs!
kind
kinds
awesome
awesome.
23 32


['i',
 'love',
 'i',
 'have',
 'tried',
 'half',
 'their',
 'and',
 'have',
 'yet',
 'to',
 'be',
 'you',
 'need',
 'to',
 'be',
 'to',
 'love',
 'these',
 'they',
 'are',
 'all',
 'of']

In [66]:
def deepCONN(user_shape,item_shape):
    model1 = Sequential()
    model1.add (Conv2D(32, (2,user_shape[1]), padding='same',
                 input_shape=user_shape, activation="relu"))
    model1.add(MaxPooling2D(pool_size=(user_shape[0],1)))     
    model1.add(Flatten())
    model1.add(Dense(100,activation="relu"))
    model1.add(BatchNormalization())
                     
                     
    model2 = Sequential()
    model2.add (Conv2D(32, (2,item_shape[1]), padding='same',
                 input_shape=item_shape, activation="relu"))
    model2.add(MaxPooling2D(pool_size=(item_shape[0],1)))
    model2.add(Flatten())
    model2.add(Dense(100,activation="relu"))
    model2.add(BatchNormalization())
                     
    model = Sequential()
    model.add(Merge([model1,model2],mode='concat'))
    model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model
    

In [100]:
model = deepCONN((50,64,1),(50,64,1))

c:\python3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [103]:
user_emb, item_emb = {},{}
for user in rev_u:
    user_emb[user] = preprocess(" ".join([rev_u[user][item] for item in rev_u[user] ]),50)
for item in rev_i:
    item_emb[item] = preprocess(" ".join([rev_i[item][user] for user in rev_i[item]]),50)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
